## Simulation Ramp (1.6 -> 1.8)

In [1]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

p_0 = 1.6;
p_1 = 1.8;
t_max  = 5.0;
p_ramp = 7.5e4;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.0_P_1.6_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.cell();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
notice(2): Group "rigid_center" created containing 1000 particles


In [2]:
#----Integrate using NPT

press_1 = hoomd.variant.linear_interp(points = [(0.0,p_0), (p_ramp, p_1)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 1.0, tauP = 12.0, P = press_1);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 
hoomd.run(p_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 830091 / 905000 | TPS 9.06691 | ETA 02:17:41
Time 00:00:20 | Step 830222 / 905000 | TPS 13.0527 | ETA 01:35:28
Time 00:00:30 | Step 830376 / 905000 | TPS 15.3562 | ETA 01:20:59
Time 00:00:40 | Step 830524 / 905000 | TPS 14.7984 | ETA 01:23:52
Time 00:00:50 | Step 830652 / 905000 | TPS 12.7962 | ETA 01:36:50
Time 00:01:00 | Step 830777 / 905000 | TPS 12.4525 | ETA 01:39:20
Time 00:01:10 | Step 830916 / 905000 | TPS 13.7369 | ETA 01:29:53
Time 00:01:20 | Step 831073 / 905000 | TPS 15.6033 | ETA 01:18:57
Time 00:01:30 | Step 831210 / 905000 | TPS 13.6343 | ETA 01:30:12
Time 00:01:40 | Step 831316 / 905000 | TPS 10.5841 | ETA 01:56:01
Time 00:01:50 | Step 831416 / 905000 | TPS 9.96402 | ETA 02:03:04
Time 00:02:00 | Step 831508 / 9

**ERROR**: Particle with unique tag 76 is no longer in the simulation box.

**ERROR**: Cartesian coordinates: 
**ERROR**: x: -52.3565 y: -2.9881 z: -8.77198
**ERROR**: Fractional coordinates: 
**ERROR**: f.x: -1.8468 f.y: 0.366063 f.z: 0.106809
**ERROR**: Local box lo: (-11.1548, -11.1548, -11.1548)
**ERROR**:           hi: (11.1548, 11.1548, 11.1548)


RuntimeError: Error computing cell list

In [ ]:
system.get_metadata()